In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from ast import literal_eval
import matplotlib.pyplot as plt

In [ ]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

transactions_train['customer_max_tran']  = transactions_train.\
groupby(['customer_id'])['t_dat_datetime'].transform('max')

In [ ]:
cust_data = transactions_train[(transactions_train['customer_max_tran'].dt.year==2020)&
                                     (transactions_train['customer_max_tran'].dt.month == 9)]

cust_data['article_id'] = 'a'+cust_data['article_id'].astype('str')

In [ ]:
cust_data['product_purchase'] = cust_data.groupby(['article_id'])['customer_id'].transform('nunique')

In [ ]:
cust_data = cust_data[cust_data['product_purchase'] >= 12]

In [ ]:
cust_data['customer_visits'] = cust_data.groupby(['customer_id'])['t_dat'].transform('nunique')

In [ ]:
cust_data = cust_data[cust_data['customer_visits'] >= 12]

In [ ]:
cust_data['customer_prod_count'] = cust_data.groupby(['customer_id'])['article_id'].transform('nunique')

In [ ]:
cust_data = cust_data[cust_data['customer_prod_count'] >= 4]

In [ ]:
cust_data['product_max_purc_date'] = cust_data.groupby(['article_id'])['t_dat_datetime'].transform('max')

In [ ]:
cust_data_2020_9 = cust_data[(cust_data['product_max_purc_date'].dt.year==2020)&
                                     (cust_data['product_max_purc_date'].dt.month == 9)]

In [ ]:
cust_data_2020_9 = cust_data_2020_9.groupby(['customer_id','article_id']).agg(tran_count = ('t_dat_datetime','nunique')).\
assign(Rank = lambda x : x.groupby('customer_id')['tran_count'].rank(method='first', ascending=False)
      ).\
reset_index().\
query('Rank <= 12').\
groupby('customer_id').agg(prods = ('article_id','unique')).reset_index().\
assign(prods = lambda x:  list(x['prods']))


# cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 

In [ ]:
cust_data_2020_9['prods_len'] = cust_data_2020_9.apply(lambda x: len(x.prods) , axis =1)


In [ ]:
article_df =  pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customer_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

article_df['article_id'] = 'a'+article_df['article_id'].astype('str')

In [ ]:
cust_data = pd.merge(cust_data,
                     cust_data_2020_9['customer_id'],
                     left_on = 'customer_id',
                     right_on = 'customer_id',
                    )

In [ ]:
cust_data.shape

In [ ]:
cust_data = pd.merge(pd.merge(cust_data ,
         article_df[['article_id','product_group_name']],
         left_on = 'article_id',
         right_on = 'article_id'
        ),
         customer_df[['customer_id','age']],
         left_on = 'customer_id',
         right_on = 'customer_id'
        )

In [ ]:
cust_features = cust_data.groupby(['customer_id','age']).agg( tran_count = ('t_dat','nunique'),
                                       article_count = ('article_id','nunique'),
                                       spend = ('price','sum')
                                      ).reset_index()

In [ ]:
cust_features_1 = pd.pivot_table(cust_data[['customer_id','product_group_name','t_dat']],
         index = 'customer_id',
         columns = 'product_group_name',
         values = 't_dat',
         aggfunc = 'nunique'
        ).fillna(0).reset_index()

In [ ]:
cust_features = pd.merge(cust_features, 
                         cust_features_1,
                         left_on = 'customer_id',
                         right_on = 'customer_id',
                        )

In [ ]:
# cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 
# cust_data_2020_9["prods"].values[:5]

In [ ]:
model_data = pd.merge(cust_data_2020_9,
                      cust_features,
                      left_on = 'customer_id',
                      right_on = 'customer_id'
                     )

In [ ]:
import gc

del cust_data_2020_9
gc.collect()

del cust_features
gc.collect()

In [ ]:
del customer_df
gc.collect()


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [ ]:
genre_mlb = mlb.fit_transform(model_data['prods'])
genre_mlb

In [ ]:
genre = mlb.inverse_transform(genre_mlb[0:1])
genre

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

In [ ]:
X_train = model_data.iloc[0:10000,2:]
y_train = genre_mlb[0:10000]
X_test = model_data.iloc[10000:11000,2:]
y_test = genre_mlb[10000:11000]

In [ ]:
model = Sequential()
model.add(Dense(5000, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(600, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

In [ ]:
X_train.columns

In [ ]:
# def macro_soft_f1(y, y_hat):
#     """Compute the macro soft F1-score as a cost.
#     Average (1 - soft-F1) across all labels.
#     Use probability values instead of binary predictions.
    
#     Args:
#         y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
#         y_hat (float32 Tensor): probability matrix of shape (BATCH_SIZE, N_LABELS)
        
#     Returns:
#         cost (scalar Tensor): value of the cost function for the batch
#     """
    
#     y = tf.cast(y, tf.float32)
#     y_hat = tf.cast(y_hat, tf.float32)
#     tp = tf.reduce_sum(y_hat * y, axis=0)
#     fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
#     fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
#     soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
#     cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
#     macro_cost = tf.reduce_mean(cost) # average on all labels
    
#     return macro_cost

In [ ]:
# def macro_f1(y, y_hat, thresh=0.7):
#     """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
#     Args:
#         y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
#         y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
#         thresh: probability value above which we predict positive
        
#     Returns:
#         macro_f1 (scalar Tensor): value of macro F1 for the batch
#     """
#     y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
#     tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
#     fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
#     fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
#     f1 = 2*tp / (2*tp + fn + fp + 1e-16)
#     macro_f1 = tf.reduce_mean(f1)
#     return macro_f1

In [ ]:
correct_prediction = tf.equal(tf.round(tf.nn.sigmoid(pred)), tf.round(y_test))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
all_labels_true = tf.reduce_min(tf.cast(correct_prediction, tf.float32),1)
accuracy2 = tf.reduce_mean(all_labels_true)

In [ ]:
def accuracy2(y, y_hat):
    
    correct_prediction = tf.equal(tf.round(tf.nn.sigmoid(y_hat)), tf.round(y))
   
    all_labels_true = tf.reduce_min(tf.cast(correct_prediction, tf.float32),1)
    
    accuracy2 = tf.reduce_mean(all_labels_true)
    
    return accuracy2

In [ ]:
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss=macro_soft_f1,
#   metrics=[macro_f1], optimizer=sgd)

model.compile(
    loss="binary_crossentropy", optimizer="sgd", metrics=[accuracy2]
)

model.fit(X_train, y_train, epochs=1, batch_size=100)

In [ ]:
preds = model.predict(X_test)
preds[preds>=0.7] = 1
preds[preds<0.7] = 0
# score = compare preds and y_test

In [ ]:
len(preds[0])

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=10)
score

In [ ]:
genre_mlb[0:1].shape

In [ ]:
preds[0:1].astype(int).shape

In [ ]:
mlb.inverse_transform(genre_mlb[0:1])

In [ ]:
mlb.inverse_transform(preds[0:1].astype(int))

In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

submission['prediction'] = top_12_products.article_id.sum()

submission.to_csv('submission.csv', index = False)